In [13]:
import cobra
import re
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
from metnet import pathway 
import sys
sys.path.append(r'./')
from COBRApy_ECM_funcation import *
from metnet import sbml_excel
import os
from copy import copy, deepcopy
import numpy as np
from cobra.io import write_sbml_model,save_json_model
from ec_code.cobrapy_ec_model_function import convert_to_irreversible,isoenzyme_split

In [14]:
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model

In [17]:
model_origin_name='./new_idea_MVA_MEP/model/iML1515_new.json'
model_new='./new_idea_MVA_MEP/model/iML1515_MVA.json'
model_new2='./new_idea_MVA_MEP/model/iML1515_MEP.json'
model_ab_json='./new_idea_MVA_MEP/model/iML1515_MVA_MEP.json'

## model_info

### A

In [ ]:
model = cobra.io.load_json_model(model_origin_name)

In [ ]:
model.medium

In [ ]:
model.reactions.get_by_id('EX_glc__D_e').bounds=(-10,1000)

In [ ]:
h2o_c=model.metabolites.get_by_id('h2o_c')
accoa_c=model.metabolites.get_by_id('accoa_c')
aacoa_c=model.metabolites.get_by_id('aacoa_c')
coa_c=model.metabolites.get_by_id('coa_c')
h_c=model.metabolites.get_by_id('h_c')
nadph_c=model.metabolites.get_by_id('nadph_c')
nadp_c=model.metabolites.get_by_id('nadp_c')
atp_c=model.metabolites.get_by_id('atp_c')
adp_c=model.metabolites.get_by_id('adp_c')

In [ ]:
HMG_CoA_c=Metabolite('HMG_CoA_c',
                formula='C27H39N7O20P3S',
                name='(S)-3-hydroxy-3-methylglutaryl-CoA',
                compartment='c'
                )

In [ ]:
mevR_c=Metabolite('mevR_c',
                formula='C6H11O4',
                name='(R)-mevalonate',
                compartment='c'
                )
mevR_e=Metabolite('mevR_e',
                formula='C6H11O4',
                name='(R)-mevalonate',
                compartment='e'
                )

In [ ]:
coa_c

In [ ]:
mvaS=Reaction("mvaS")
mvaS.name='hydroxymethylglutaryl-CoA synthase'
mvaS.subsystem='unknow'
mvaS.lower_bound=-1000
mvaS.upper_bound=1000
mvaS.add_metabolites({
    accoa_c:-1.0,
    h2o_c:-1.0,
    aacoa_c:-1.0,
    HMG_CoA_c:1.0,
    coa_c:1.0,
    h_c:1.0
})
model.add_reactions([mvaS])

In [ ]:
mvaE=Reaction("mvaE")
mvaE.name='acetyl-CoA acetyltransferase/HMG-CoA reductase'
mvaE.subsystem='unknow'
mvaE.lower_bound=-1000
mvaE.upper_bound=1000
mvaE.add_metabolites({
    HMG_CoA_c:-1.0,
    nadph_c:-2.0,
    h_c:-2.0,
    nadp_c:2.0,
    coa_c:1.0,
    mevR_c:1.0
})
model.add_reactions([mvaE])

In [ ]:
EX_mevR_e=Reaction("EX_mevR_e")
EX_mevR_e.name='(R)-mevalonate exchange'
EX_mevR_e.subsystem='unknow'
EX_mevR_e.lower_bound=0
EX_mevR_e.upper_bound=1000
EX_mevR_e.add_metabolites({
    mevR_e:-1.0
})
mevR_c2e=Reaction("mevR_c2e")
mevR_c2e.name='(R)-mevalonate c2e'
mevR_c2e.subsystem='unknow'
mevR_c2e.lower_bound=-1000
mevR_c2e.upper_bound=1000
mevR_c2e.add_metabolites({
    mevR_c:-1.0,
    mevR_e:1.0
})
model.add_reactions([EX_mevR_e])
model.add_reactions([mevR_c2e])

In [ ]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [ ]:
# mev5p_c=Metabolite('mev5p_c',
#                 formula='C6H10O7P',
#                 name='(R)-5-phosphomevalonate,mevalonate-P',
#                 compartment='c'
#                 )
# mev5pp_c=Metabolite('mev5pp_c',
#                 formula='C6H10O10P2',
#                 name='(R)-mevalonate diphosphate,mevalonate-5-PP',
#                 compartment='c'
#                 )
# ERG12=Reaction("ERG12")
# ERG12.name='mevalonate kinase'
# ERG12.subsystem='unknow'
# ERG12.lower_bound=-1000
# ERG12.upper_bound=1000
# ERG12.add_metabolites({
#     atp_c:-1.0,
#     mevR_c:-1.0,
#     mev5p_c:1.0,
#     h_c:1.0,
#     adp_c:1.0
# })
# model.add_reactions([ERG12])
# ERG8=Reaction("ERG8")
# ERG8.name='phosphomevalonate kinase'
# ERG8.subsystem='unknow'
# ERG8.lower_bound=-1000
# ERG8.upper_bound=1000
# ERG8.add_metabolites({
#     mev5p_c:-1.0,
#     atp_c:-1.0,
#     mev5pp_c:1.0,
#     adp_c:1.0
# })
# model.add_reactions([ERG8])
# ipdp_c=model.metabolites.get_by_id('ipdp_c')
# pi_c=model.metabolites.get_by_id('pi_c')
# co2_c=model.metabolites.get_by_id('co2_c')
# MVD1=Reaction("MVD1")
# MVD1.name='phosphomevalonate kinase'
# MVD1.subsystem='unknow'
# MVD1.lower_bound=0
# MVD1.upper_bound=1000
# MVD1.add_metabolites({
#     mev5pp_c:-1.0,
#     atp_c:-1.0,
#     co2_c:1.0,
#     adp_c:1.0,
#     ipdp_c:1.0,
#     pi_c:1.0
# })
# model.add_reactions([MVD1])

In [ ]:
model.objective=model.reactions.EX_mevR_e

In [ ]:
model.optimize()
model.summary()

In [ ]:
save_json_model(model,model_new)
write_sbml_model(model,model_new.split('.json')[0]+'.xml')
sbml_excel(model_new.split('.json')[0]+'.xml',model_new2.split('.json')[0]+'.xlsx')

### B

In [ ]:
model = cobra.io.load_json_model(model_origin_name)
model.medium

In [ ]:
mevR_c=Metabolite('mevR_c',
                formula='C6H11O4',
                name='(R)-mevalonate',
                compartment='c'
                )
mevR_e=Metabolite('mevR_e',
                formula='C6H11O4',
                name='(R)-mevalonate',
                compartment='e'
                )
mev5p_c=Metabolite('mev5p_c',
                formula='C6H10O7P',
                name='(R)-5-phosphomevalonate,mevalonate-P',
                compartment='c'
                )
mev5pp_c=Metabolite('mev5pp_c',
                formula='C6H10O10P2',
                name='(R)-mevalonate diphosphate,mevalonate-5-PP',
                compartment='c'
                )

In [ ]:
EX_mevR_e=Reaction("EX_mevR_e")
EX_mevR_e.name='(R)-mevalonate exchange'
EX_mevR_e.subsystem='unknow'
EX_mevR_e.lower_bound=-1000
EX_mevR_e.upper_bound=1000
EX_mevR_e.add_metabolites({
    mevR_e:-1.0
})
mevR_c2e=Reaction("mevR_c2e")
mevR_c2e.name='(R)-mevalonate c2e'
mevR_c2e.subsystem='unknow'
mevR_c2e.lower_bound=-1000
mevR_c2e.upper_bound=1000
mevR_c2e.add_metabolites({
    mevR_c:-1.0,
    mevR_e:1.0
})
model.add_reactions([EX_mevR_e])
model.add_reactions([mevR_c2e])

In [ ]:
ERG12=Reaction("ERG12")
ERG12.name='mevalonate kinase'
ERG12.subsystem='unknow'
ERG12.lower_bound=-1000
ERG12.upper_bound=1000
ERG12.add_metabolites({
    atp_c:-1.0,
    mevR_c:-1.0,
    mev5p_c:1.0,
    h_c:1.0,
    adp_c:1.0
})
model.add_reactions([ERG12])

In [ ]:
ERG8=Reaction("ERG8")
ERG8.name='phosphomevalonate kinase'
ERG8.subsystem='unknow'
ERG8.lower_bound=-1000
ERG8.upper_bound=1000
ERG8.add_metabolites({
    mev5p_c:-1.0,
    atp_c:-1.0,
    mev5pp_c:1.0,
    adp_c:1.0
})
model.add_reactions([ERG8])

In [ ]:
ipdp_c=model.metabolites.get_by_id('ipdp_c')
pi_c=model.metabolites.get_by_id('pi_c')
co2_c=model.metabolites.get_by_id('co2_c')
ppi_c=model.metabolites.get_by_id('ppi_c')

In [ ]:
MVD1=Reaction("MVD1")
MVD1.name='phosphomevalonate kinase'
MVD1.subsystem='unknow'
MVD1.lower_bound=0
MVD1.upper_bound=1000
MVD1.add_metabolites({
    mev5pp_c:-1.0,
    atp_c:-1.0,
    co2_c:1.0,
    adp_c:1.0,
    ipdp_c:1.0,
    pi_c:1.0
})
model.add_reactions([MVD1])

In [ ]:
ipdp_c

In [ ]:
ipal_c=Metabolite('ipal_c',
                formula='C5H10O',
                name='isopentenyl alcohol/isoprenol',
                compartment='c'
                )
ipal_e=Metabolite('ipal_e',
                formula='C5H10O',
                name='isopentenyl alcohol/isoprenol',
                compartment='e'
                )

In [ ]:
Nudf=Reaction("Nudf")
Nudf.name='phosphatase'
Nudf.subsystem='unknow'
Nudf.lower_bound=0
Nudf.upper_bound=1000
Nudf.add_metabolites({
    ipdp_c:-1.0,
    h2o_c:-1.0,
    ppi_c:1.0,
    ipal_c:1.0,
    h_c:1.0
})
model.add_reactions([Nudf])

In [ ]:
EX_ipal_e=Reaction("EX_ipal_e")
EX_ipal_e.name='isoprenol exchange'
EX_ipal_e.subsystem='unknow'
EX_ipal_e.lower_bound=0
EX_ipal_e.upper_bound=1000
EX_ipal_e.add_metabolites({
    ipal_e:-1.0
})
ipal_c2e=Reaction("ipal_c2e")
ipal_c2e.name='isoprenol c2e'
ipal_c2e.subsystem='unknow'
ipal_c2e.lower_bound=-1000
ipal_c2e.upper_bound=1000
ipal_c2e.add_metabolites({
    ipal_c:-1.0,
    ipal_e:1.0
})
model.add_reactions([EX_ipal_e])
model.add_reactions([ipal_c2e])

In [ ]:
model.reactions.get_by_id('DMPPS').bounds=(0,0)

In [ ]:
model.reactions.get_by_id('IPDPS').bounds=(0,0)

In [ ]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds=(0.1,1000)

In [ ]:
model.objective=model.reactions.EX_ipal_e

In [ ]:
model.reactions.EX_mevR_e

In [ ]:
model.optimize()
model.summary()

In [ ]:
save_json_model(model,model_new2)
write_sbml_model(model,model_new2.split('.json')[0]+'.xml')
sbml_excel(model_new2.split('.json')[0]+'.xml',model_new2.split('.json')[0]+'.xlsx')

### AB

In [ ]:
model_a_name=model_new
model_b_name=model_new2
model_a=load_json_model(model_a_name)
model_b=load_json_model(model_b_name)

In [ ]:
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )

In [ ]:
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name

In [ ]:
for i in model_t.reactions:
    model.add_reaction(i)

In [ ]:
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])

In [ ]:
model_origin_name='./new_idea_MVA_MEP/model/iML1515_new.json'
origin_model=cobra.io.load_json_model(model_origin_name)
# origin_model.reactions.get_by_id('EX_xyl__D_e').bounds=(-10.0,1000)
origin_model.medium

In [ ]:
for i in origin_model.medium:
    model.add_reaction(origin_model.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
#model.medium

In [ ]:
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium

In [ ]:
model.reactions.B_mevR_e_con_2.bounds=(-1000,1000)

In [ ]:
model.add_reaction(model_b.reactions.EX_ipal_e)

In [ ]:
model.reactions.EX_ipal_e

In [ ]:
model.reactions.EX_ipal_e.bounds=(0,1000)

In [ ]:
model.medium

In [ ]:
model.objective=model.reactions.get_by_id('EX_ipal_e')

In [ ]:
model.optimize()
model.summary()

In [ ]:
save_json_model(model,model_ab_json)

### new_idea

In [ ]:
from new_cobra import *

In [ ]:
def Get_Model_Data(model):
    """Returns reaction_list,metabolite_list,lb_list,ub_list,coef_matrix from model.
    
    Notes: 
    ----------
    *model： is in SBML format (.xml).
    """
    reaction_list=[]
    reaction_list_A=[]
    reaction_list_B=[]
    metabolite_list=[]
    lb_list={}
    ub_list={}
    coef_matrix={}
    for rea in model.reactions:
        reaction_list.append(rea.id)
        if rea.id.startswith('A_'):
            reaction_list_A.append(rea.id)
        if rea.id.startswith('B_'):
            reaction_list_B.append(rea.id)    
        lb_list[rea.id]=rea.lower_bound
        ub_list[rea.id]=rea.upper_bound
        for met in model.metabolites:
            metabolite_list.append(met.id)
            try:
                rea.get_coefficient(met.id)  
            except:
                pass
            else:
                coef_matrix[met.id,rea.id]=rea.get_coefficient(met.id)
    reaction_list=list(set(reaction_list))
    metabolite_list=list(set(metabolite_list))
    return(reaction_list,reaction_list_A,reaction_list_B,metabolite_list,lb_list,ub_list,coef_matrix)

In [ ]:
def Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,target):
    Concretemodel_Need_Data={}
    # reaction_g0=pd.read_csv(reaction_g0_file,index_col=0,sep='\t')
    # Concretemodel_Need_Data['reaction_g0']=reaction_g0
    # metabolites_lnC = pd.read_csv(metabolites_lnC_file, index_col=0, sep='\t')
    # Concretemodel_Need_Data['metabolites_lnC']=metabolites_lnC
    model=cobra.io.load_json_model(model_file)
    try:
        product = model.metabolites.get_by_id(target)
        model.add_boundary(product, type='demand')  # add demand reaction as the objective
    except:
        pass
    cobra.manipulation.modify.convert_to_irreversible(model)
    isoenzyme_split(model)
    reaction_kcat_MW=pd.read_csv(reaction_kcat_MW_file,index_col=0)
    Concretemodel_Need_Data['model']=model
    Concretemodel_Need_Data['reaction_kcat_MW']=reaction_kcat_MW
    [reaction_list,reaction_list_A,reaction_list_B,metabolite_list,lb_list,ub_list,coef_matrix]=Get_Model_Data(model)
    Concretemodel_Need_Data['reaction_list']=reaction_list
    Concretemodel_Need_Data['reaction_list_A']=reaction_list_A
    Concretemodel_Need_Data['reaction_list_B']=reaction_list_B
    Concretemodel_Need_Data['metabolite_list']=metabolite_list
    Concretemodel_Need_Data['lb_list']=lb_list
    Concretemodel_Need_Data['ub_list']=ub_list
    Concretemodel_Need_Data['coef_matrix']=coef_matrix
    return (Concretemodel_Need_Data)

In [ ]:
def Template_Concretemodel2(reaction_list=None,metabolite_list=None,coef_matrix=None,reaction_kcat_MW=None,lb_list=None,ub_list=None,reaction_list_A=None,reaction_list_B=None,\
    set_substrate_ini=False,substrate_name=None,substrate_value=None,substrates=None,substrates_bool=False,\
    set_biomass_ini=False,biomass_value=None,biomass_list=None,biomass_id=None,set_A_biomass_ini=None,set_B_biomass_ini=None,\
    set_bound=False,set_stoi_matrix=False,set_enzyme_constraint=False,set_part_enzyme_constraint=False,set_obj_single_E_value=False,E_total=None,\
    obj_name=None,obj_target=None,set_obj_value=False,set_target_ini=False,target_value=None,\
    set_obj_V_value=False,set_new_stoi_matrix=False,set_obj_E_value=False,\
    set_ratio=False,set_ratio_bound=False,set_pass_ini=False,set_pass_value=None,\
    set_metabolite=False,set_obj_Met_value=False,metabolites_lnC=None,\
    set_sub_ratio=False,sub_a=None,sub_b=None,sub_ratio=None):
    

    Concretemodel = ConcreteModel()
    Concretemodel.reaction = pyo.Var(reaction_list,  within=NonNegativeReals)
    Concretemodel.z = pyo.Var(reaction_list,  within=pyo.Binary)
    Concretemodel.x = pyo.Var()
    Concretemodel.y = pyo.Var()

    #Set upper and lower bounds of metabolite concentration
    # if set_metabolite:
    #     def set_metabolite(m,i):
    #         return  inequality(metabolites_lnC.loc[i,'lnClb'], m.metabolite[i], metabolites_lnC.loc[i,'lnCub'])
    #     Concretemodel.set_metabolite= Constraint(metabolite_list,rule=set_metabolite)  

    if set_ratio:
        def set_ratio(m):
            return  m.x+m.y==100
        Concretemodel.set_ratio= Constraint(rule=set_ratio)    
        
    if set_ratio_bound:
        def set_ratio_bound(m):
            return inequality(1,m.x,99)
        Concretemodel.set_ratio_bound = Constraint(rule=set_ratio_bound)     

    #Minimizing the flux sum of pathway (pFBA)
    if set_obj_V_value:             
        def set_obj_V_value(m):
            return sum(m.reaction[j] for j in reaction_list)
        Concretemodel.obj = Objective(rule=set_obj_V_value, sense=minimize)  
    
    #Set the maximum flux as the object function
    if set_obj_value:   
        def set_obj_value(m):
            return m.reaction[obj_name]
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=maximize)
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=minimize)

    #Set the minimum enzyme cost of a pathway as the object function
    if set_obj_E_value:             
        def set_obj_E_value(m):
            return sum(m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index)
        Concretemodel.obj = Objective(rule=set_obj_E_value, sense=minimize)  

    #To calculate the variability of enzyme usage of single reaction.
    if set_obj_single_E_value:             
        def set_obj_single_E_value(m):
            return m.reaction[obj_name]/(reaction_kcat_MW.loc[obj_name,'kcat_MW'])
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=maximize) 
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=minimize)

    if set_sub_ratio:
        def set_sub_ratio(m):
            return m.reaction[sub_a]==4*m.reaction[sub_b]
        Concretemodel.set_sub_ratio = Constraint(rule=set_sub_ratio)

    #To calculate the concentration variability of metabolites.
    # if set_obj_Met_value:   
    #     def set_obj_Met_value(m):
    #         return m.metabolite[obj_name]
    #     if obj_target=='maximize':
    #         Concretemodel.obj = Objective(rule=set_obj_Met_value, sense=maximize)
    #     elif obj_target=='minimize':
    #         Concretemodel.obj = Objective(rule=set_obj_Met_value, sense=minimize)

    #Adding flux balance constraints （FBA）
    if set_stoi_matrix:
        def set_stoi_matrix(m,i):
            return sum(coef_matrix[i,j]*m.reaction[j]  for j in reaction_list if (i,j) in coef_matrix.keys() )==0
        Concretemodel.set_stoi_matrix = Constraint( metabolite_list,rule=set_stoi_matrix)

    #Adding flux balance constraints （newFBA）
    if set_new_stoi_matrix:
        def set_new_stoi_matrix(m,i):
            for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/m.x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/m.y)*coef_matrix[i,j]
            return sum(coef_matrix[i,j]*m.reaction[j]  for j in reaction_list if (i,j) in coef_matrix.keys() )==0
        Concretemodel.set_new_stoi_matrix = Constraint( metabolite_list,rule=set_new_stoi_matrix)

    #Adding the upper and lower bound constraints of reaction flux
    if set_bound:
        def set_bound(m,j):
            return inequality(lb_list[j],m.reaction[j],ub_list[j])
        Concretemodel.set_bound = Constraint(reaction_list,rule=set_bound) 

    #Set the upper bound for substrate input reaction flux
    if set_substrate_ini and substrates_bool==False:
        def set_substrate_ini(m): 
            return m.reaction[substrate_name] <= substrate_value
        Concretemodel.set_substrate_ini = Constraint(rule=set_substrate_ini)  
    
     #Set the upper bound for substrate input reaction flux
    if set_substrate_ini and substrates_bool==True:
        def set_substrates_ini(m,j): 
            return m.reaction[j]<= substrates[j]
        Concretemodel.set_substrates_ini = Constraint(substrates.keys(),rule=set_substrates_ini)  
    
    if set_biomass_ini:
        def set_biomass_ini(m): 
            return m.reaction[biomass_id] >=biomass_value
        Concretemodel.set_biomass_ini = Constraint(rule=set_biomass_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_A_biomass_ini:
        def set_A_biomass_ini(m): 
            return m.reaction['A_'+biomass_id] >=biomass_list['A']
        Concretemodel.set_A_biomass_ini = Constraint(rule=set_A_biomass_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_B_biomass_ini:
        def set_B_biomass_ini(m): 
            return m.reaction['B_'+biomass_id] >=biomass_list['B']
        Concretemodel.set_B_biomass_ini = Constraint(rule=set_B_biomass_ini)  
    
    #Set the lower bound for target synthesis reaction flux
    if set_target_ini:
        def set_target_ini(m): 
            return m.reaction[obj_name] ==target_value
        Concretemodel.set_target_ini = Constraint(rule=set_target_ini)  

    #Adding enzymamic constraints
    if set_enzyme_constraint:
        def set_enzyme_constraint(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list)<= E_total
        Concretemodel.set_enzyme_constraint = Constraint(rule=set_enzyme_constraint)    
    
    #Adding Part A enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_A(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_A)<= E_total
        Concretemodel.set_enzyme_constraint_A = Constraint(rule=set_enzyme_constraint_A)

    #Adding Part B enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_B(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_B)<= E_total
        Concretemodel.set_enzyme_constraint_B = Constraint(rule=set_enzyme_constraint_B)

    #Adding pass meta flux constraints
    if set_pass_ini:
        def set_pass_ini(m,j):
            return m.reaction[j] >= set_pass_value[j]
        Concretemodel.set_pass_ini = Constraint(set_pass_value.keys(),rule=set_pass_ini)  

    return Concretemodel

In [ ]:
def Model_Solve(model,solver):
    opt = pyo.SolverFactory(solver)
    opt.solve(model)
    return model

In [ ]:
def ECM_FBA(a,x,y,mid_a,mid_b,bio_a,bio_b):
    #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
    #substrates={'EX_glc__D_e_reverse':10}
    substrates={'EX_glc__D_e_reverse':a}
    substrate_value=10
    substrate_name='EX_glc__D_e_reverse'
    E_total=0.227
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    if len(substrates.keys())==1:
            for i in substrates.keys():
                ub_list[i]=substrates[i]
                substrate_name=i
                substrate_value=substrates[i]
            substrates_bool=False
    else:
            for i in substrates.keys():
                    ub_list[i]=substrates[i]
            substrate_name=''
            substrate_value=0
            substrates_bool=True
    
    obj_name='EX_ipal_e'
    obj_target='maximize'
    biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
    #pass_value={'A_tyr__L_e_con_1':tyr,'B_phpyr_e_con_2_reverse':php}
    EcoECM_FBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,\
        obj_name=obj_name,obj_target=obj_target,set_obj_value=True,biomass_id=biomass_id,set_substrate_ini=True,substrate_name=substrate_name,\
        substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,E_total=E_total,\
        set_enzyme_constraint=False,set_part_enzyme_constraint=True,set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_list={'A':bio_a,'B':bio_b},\
        set_pass_ini=False,set_pass_value=None,set_sub_ratio=False,sub_a=sub_a,sub_b=sub_b,sub_ratio=ratio)
    opt_ecm_FBA=Model_Solve(EcoECM_FBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_FBA

In [ ]:
def ECM_pFBA(a,x,y,obj,mid_a,mid_b,bio_a,bio_b):
    #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
    #substrates={'EX_glc__D_e_reverse':10}
    substrates={'EX_glc__D_e_reverse':a}
    substrate_value=10
    substrate_name=''
    E_total=0.227
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    if len(substrates.keys())==1:
            for i in substrates.keys():
                ub_list[i]=substrates[i]
                substrate_name=i
                substrate_value=substrates[i]
            substrates_bool=False
    else:
            for i in substrates.keys():
                    ub_list[i]=substrates[i]
            substrate_name=''
            substrate_value=0
            substrates_bool=True
    substrates_bool=True
    obj_name='EX_ipal_e'
    obj_target='maximize'
    biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
    #pass_value={'A_tyr__L_e_con_1':tyr,'B_phpyr_e_con_2_reverse':php}
    EcoECM_pFBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,\
        obj_name=obj_name,obj_target=obj_target,set_obj_value=False,set_target_ini=True,set_obj_V_value=True,target_value=obj,\
        biomass_id=biomass_id,set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_list={'A':bio_a,'B':bio_b},set_substrate_ini=True,substrate_name=substrate_name,\
        substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,E_total=E_total,\
        set_enzyme_constraint=False,set_part_enzyme_constraint=True,set_pass_ini=False,set_pass_value=None,set_sub_ratio=False,sub_a=sub_a,sub_b=sub_b,sub_ratio=ratio)
    opt_ecm_pFBA=Model_Solve(EcoECM_pFBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_pFBA

In [ ]:
def ECM_FBA_knock_out(a,x,y,mid_a,mid_b,bio_a,bio_b,rea):
    #EcoECM=EcoECM(Concretemodel_Need_Data,'DM_B_C06104','maximize',substrates,E_total)\
    #substrates={'EX_glc__D_e_reverse':10}
    substrates={'EX_glc__D_e_reverse':a}
    substrate_value=10
    substrate_name='EX_glc__D_e_reverse'
    E_total=0.227
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    lb_list[rea]=0
    ub_list[rea]=0
    if len(substrates.keys())==1:
            for i in substrates.keys():
                ub_list[i]=substrates[i]
                substrate_name=i
                substrate_value=substrates[i]
            substrates_bool=False
    else:
            for i in substrates.keys():
                    ub_list[i]=substrates[i]
            substrate_name=''
            substrate_value=0
            substrates_bool=True
    
    obj_name='EX_ipal_e'
    obj_target='maximize'
    biomass_id='BIOMASS_Ec_iML1515_core_75p37M'
    #pass_value={'A_tyr__L_e_con_1':tyr,'B_phpyr_e_con_2_reverse':php}
    EcoECM_FBA=Template_Concretemodel2(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,\
        obj_name=obj_name,obj_target=obj_target,set_obj_value=True,biomass_id=biomass_id,set_substrate_ini=True,substrate_name=substrate_name,\
        substrate_value=substrate_value,substrates=substrates,substrates_bool=substrates_bool,set_stoi_matrix=True,set_bound=True,E_total=E_total,\
        set_enzyme_constraint=False,set_part_enzyme_constraint=True,set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_list={'A':bio_a,'B':bio_b},\
        set_pass_ini=False,set_pass_value=None,set_sub_ratio=False,sub_a=sub_a,sub_b=sub_b,sub_ratio=ratio)
    opt_ecm_FBA=Model_Solve(EcoECM_FBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_FBA

In [ ]:
def get_two_float(f_str, n):
    a, b, c = str(f_str).partition('.')
    c = c[:n]
    return float(".".join([a, c]))

In [ ]:
def unpdate_kcat_mw(reaction_list,kcat_mw,new_file):
    reaction_up_kact_MW=pd.DataFrame()
    for i in reaction_list:
        if i.startswith('A_'):
            id=i[2:]
            if id in kcat_mw.index:
                reaction_up_kact_MW.at[i,'kcat_MW']=kcat_mw.at[id,'kcat_MW']
        if i.startswith('B_'):
            id=i[2:]
            if id in kcat_mw.index:
                reaction_up_kact_MW.at[i,'kcat_MW']=kcat_mw.at[id,'kcat_MW'] 
    reaction_up_kact_MW.to_csv(new_file)
    return reaction_up_kact_MW

## ECM

In [15]:
from double_ECM import * 

In [16]:
reaction_kcat_MW_file='./new_idea_MVA_MEP/EC/reaction_kcat_MW_MVA_MEP_origin.csv'

In [18]:
# model_file=model_new
model_file=model_ab_json
reaction_kcat_MW_file='./new_idea_MVA_MEP/EC/reaction_kcat_MW_MVA_MEP_origin.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file,'ipal_e')

In [19]:
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_MVA_MEP/EC/reaction_kcat_MW_MVA_MEP.csv')

In [20]:
list_x=[3,2,1]
list_y=[1,2,3]

In [ ]:
Concretemodel_Need_Data['reaction_kcat_MW']=reaction_kcat_MW=pd.read_csv(reaction_kcat_MW_file,index_col=0)
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw(Concretemodel_Need_Data['reaction_list'],Concretemodel_Need_Data['reaction_kcat_MW'],'./new_idea_MVA_MEP/EC/reaction_kcat_MW_MVA_MEP.csv')

In [21]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import datetime
from thread import *
import sys
sys.path.append(r'./')

In [22]:
if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=8)
    with pool as executor:
        start = datetime.datetime.now()
        #print("pass_2")
        tmp=[]
        futures = {executor.submit(fba_max_product,Concretemodel_Need_Data,(list_x[i],list_y[i]),40,0,0,0.65,0.65,'EX_ipal_e'): i for i in range(0,len(list_x))}
        # for future in as_completed(futures):
        #     tmp.append(future.result())
    end = datetime.datetime.now()
    print (end - start)
    print(futures)

0:00:17.828403
{<Future at 0x1f3c14cdc88 state=finished returned float>: 0, <Future at 0x1f3c6d642b0 state=finished returned float>: 1, <Future at 0x1f3c6d64c18 state=finished returned float>: 2}


In [23]:
result_list_mass=pd.DataFrame()
for future in as_completed(futures):
    x=list_x[futures[future]]
    y=list_y[futures[future]]
    real=1
    real_x=int(x*real)
    real_y=int(y*real)
    print('A:B:C='+str(real_x)+':'+str(real_y))
    print(future.result())
    result_list_mass.at[str(real_x)+':'+str(real_y),'flux']=future.result()
result_list=pd.DataFrame()
for i in range(0,len(list_x)):
    real=1
    real_x=int(list_x[i]*real)
    real_y=int(list_y[i]*real)
    result_list.at[str(real_x)+':'+str(real_y),'flux']=result_list_mass.at[str(real_x)+':'+str(real_y),'flux']
result_list

A:B:C=2:2
2.4924224927342493
A:B:C=1:3
2.644729225479511
A:B:C=3:1
1.5534946709572068


,flux
3:1,1.553495
2:2,2.492422
1:3,2.644729


In [ ]:
# pass_v=0
# for i in range(0,3):
#     a=ECM_FBA(40,list_x[i],list_y[i],0,0,0.1,0.1)
#     print('obj:')
#     print(a.obj())
#     # a_obj=get_two_float(a.obj(),2)
#     # b=ECM_pFBA(40,list_x[i],list_y[i],a_obj,0,0,0.1,0.1)
#     # print('pFBA')
#     # print('glc:'+str(b.reaction['EX_glc__D_e_reverse'].value))
#     # print('glc_A:'+str(b.reaction['A_glc__D_e_con_1_reverse'].value))
#     # print('glc_B:'+str(b.reaction['B_glc__D_e_con_2'].value))
#     if pass_v>a.obj():
#         print('True')
#     else:
#         print('False')
#     pass_v=a.obj()

In [ ]:
# model_out=load_json_model(model_file)
# cobra.manipulation.modify.convert_to_irreversible(model_out)
# isoenzyme_split(model_out)
# model_pfba_solution_detail=pd.DataFrame()
# a=ECM_FBA(100,1,3,0,0,0.1,0.1)
# print('obj:')
# print(a.obj())
# a_obj=get_two_float(a.obj(),2)
# b=ECM_pFBA(100,1,3,a_obj,0,0,0.1,0.1)
# reaction_kcat_mw=Concretemodel_Need_Data['reaction_kcat_MW']
# print('pFBA')
# print('glc:'+str(b.reaction['EX_glc__D_e_reverse'].value))
# print('glc_A:'+str(b.reaction['A_glc__D_e_con_1_reverse'].value))
# print('glc_B:'+str(b.reaction['B_glc__D_e_con_2'].value))
# for i in b.reaction:
#     model_pfba_solution_detail.loc[i,'fluxes'] = b.reaction[i].value
#     if i=='DM_ipal_e':
#         continue
#     reaction_detail=model_out.reactions.get_by_id(i)
#     if i in reaction_kcat_mw.index:
#             # model_pfba_solution_detail.loc[i,
#             #                                'kcat'] = reaction_kcat_mw.loc[i, 'kcat']
#             # model_pfba_solution_detail.loc[i,
#             #                                'MW'] = reaction_kcat_mw.loc[i, 'MW']
#             model_pfba_solution_detail.loc[i,
#                                            'kcat_MW'] = reaction_kcat_mw.loc[i, 'kcat_MW']
#             model_pfba_solution_detail.loc[i,
#                                            'enz'] = model_pfba_solution_detail.loc[i, 'fluxes']/model_pfba_solution_detail.loc[i, 'kcat_MW']
#             # if 'source' in reaction_kcat_mw.columns:
#             #     model_pfba_solution_detail.loc[i,
#             #                                    'source'] = reaction_kcat_mw.loc[i, 'source']
#     model_pfba_solution_detail.loc[i, 'equ'] = reaction_detail.reaction
# if pass_v>a.obj():
#     print('True')
# else:
#     print('False')
# pass_v=a.obj()
# model_pfba_solution_detail.to_csv('./new_idea_MVA_MEP/result/result_pfba_enz.csv')